# Data Preparation for Model Building

This notebook prepares two dataframes, `df_er` and `df_rest`, for the rest of the modeling process. It involves reading multiple data files, cleaning, merging, aggregating, and preparing the target variable.

## Importing Libraries

In [ ]:
import pandas as pd

## Loading Data

We read monthly data files for the year 2018 from January to December.

In [ ]:
# Read monthly data files
file_paths = ['data/STOCK_PROFILPRO_CLIENTS_{:02d}2018.csv'.format(i) for i in range(1, 13)]
dataframes = []
for i, file_path in enumerate(file_paths, 1):
    df = pd.read_csv(file_path, sep=';', encoding='latin-1')
    df['col'] = i  # Add a column to track the month
    dataframes.append(df)

# Concatenate all dataframes
df = pd.concat(dataframes, ignore_index=True)

We add a 'col' column to each dataframe to track the month number (from 1 to 12) before concatenating them into a single dataframe.

## Preprocessing

### Dropping Unnecessary Columns

We drop columns that are not needed for the analysis.

In [ ]:
# Drop unnecessary columns
columns_to_drop = ['seg_mark_op', 'risque_defaut', 'NSCO085', 'NSCO032', 'NSCO041', 'NSCO033', 
                   'NSCO110', 'NSCO121', 'dcl', 'rtr', 'communaute_pro', 'CBS', 'poste_banque']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

### Renaming Columns

We rename some columns for better readability.

In [ ]:
# Rename columns
df.rename(columns={
    'QPCL063': 'compte epargne',
    'QPCL064': 'compte epargne a echeance',
    'QPCL065': 'compte à terme',
    'QPCL069': 'prêt immo',
    'QPCL071': 'compte titre',
    'QPCL072': 'prêt perso',
    'QPCL074': 'crédit fonctionnement',
    'QPCL075': 'crédit équipement',
    'QPCL076': 'engagement par signature',
    'QPCL079': 'santé prévoyance',
    'QPCL080': 'dommage',
    'QPCL083': 'assurance vie',
    'QPCL089': 'Cartes de Paiement',
    'QPCL373': 'Esprit Libre PRO',
    'QPCL142': 'BNP NET Pro et Evolution',
    'QPCL165': 'Contrat Commerçant'
}, inplace=True)

## Filtering Data

We select rows where `NANCCLI == 2`.

In [ ]:
# Filter data where NANCCLI == 2
df_pro = df[df['NANCCLI'] == 2]

## Removing Duplicates

We sort the dataframe and remove duplicates, keeping the last occurrence based on 'CCLI' and 'CCLIKPI'.

In [ ]:
# Sort the dataframe by 'CCLI', 'CCLIKPI', and 'col'
df_pro = df_pro.sort_values(by=['CCLI', 'CCLIKPI', 'col'])

# Drop duplicates keeping the last one based on 'CCLI' and 'CCLIKPI'
df_pro_unique = df_pro.drop_duplicates(subset=['CCLI', 'CCLIKPI'], keep='last')

## Splitting Data

We exclude rows where `CNOUVSEG == 'AS'`, then split the data into two dataframes based on `CNOUVSEG`:

- `df_pro_unique_er`: where `CNOUVSEG == 'ER'`
- `df_pro_unique_rest`: the rest of the data

In [ ]:
# Exclude rows where CNOUVSEG == 'AS'
df_pro_unique = df_pro_unique[df_pro_unique['CNOUVSEG'] != 'AS']

# Split into 'ER' and 'Rest'
df_pro_unique_er = df_pro_unique[df_pro_unique['CNOUVSEG'] == 'ER']
df_pro_unique_rest = df_pro_unique[df_pro_unique['CNOUVSEG'] != 'ER']

## Defining Priority Orders for Aggregation

We define custom aggregation functions for categorical variables where we have a priority order.

In [ ]:
# Custom aggregation function for categorical variables with priority order
def agg_priority(series, priority_order):
    for priority in priority_order:
        if priority in series.values:
            return priority
    return series.values[0]  # Fallback to the first value if no priority match found

# Wrapper function to apply the correct priority order
def agg_wrapper(column, priority_orders):
    return lambda series: agg_priority(series, priority_orders[column])

## Grouping and Aggregating Data

We define custom aggregation functions with priority orders for categorical variables and aggregate the data accordingly.

In [ ]:
# Define priority order for categorical variables
priority_orders = {
    'CNOUVSEG': ['ER'],
    'CMOTENT1': [98., 26., 44., 4., 1., 97., 28., 69., 96., 83., 7., 76., 89., 21., 49.],
    'CUODR': [8065, 8061, 8060, 8085, 8063, 8058, 8080, 8066, 8064, 8054, 8052,
              8055, 8062, 8075, 8053, 8051, 8040, 8056, 8057],
    'CENSEIGNE': [0, 1],
    'etebac': [0, 1, 2],
    'prevoyance_pro': [0, 1, 2],
    'section': ["Commerce ; réparation d'automobiles et de motocycles",
                'Construction',
                'Activités spécialisées, scientifiques et techniques',
                'Autres activités de services', 'Industrie manufacturière',
                'Hébergement et restauration',
                "Activités financières et d'assurance", 'Activités immobilières',
                'Agriculture, sylviculture et pêche', ' Pas de section',
                'Transports et entreposage',
                'Activités de services administratifs et de soutien',
                'Information et communication', 'Santé humaine et action sociale',
                'Enseignement',
                "Production et distribution d'eau ; assainissement, ges",
                "Production et distribution d'électricité, de gaz, de v",
                'Arts, spectacles et activités récréatives',
                'Industries extractives', 'Activités extra-territoriales'],
    'code_sous_marche': ['52', '45', '74', '93', '29', '55', '28', '70', '05', '51', '00',
                         '60', '20', 'COM', '72', '67', '01', '50', '85', '92', '63', 'nan',
                         '31', '22', '73', '25', '71', '80', '36', '90', '15', '65', '17',
                         '40', '21', '37', '18', '64', '32', '35', 'SRV', '24', '26', 'ART',
                         '34', 'LIB', '33', '30', '61', '19', '14', '27', 'AUT', 'INC',
                         'SAN', '99', '02'],
    'CCOTABNP': [9., 8., 7., 5., 2., 1., 'nan']
}

# Define aggregation methods for each column
agg_dict = {
    'CTYPCLI': 'mean',
    'CNOUVSEG': agg_wrapper('CNOUVSEG', priority_orders),
    'AGE': 'max',
    'NANCCLI': 'max',
    'CMOTENT1': agg_wrapper('CMOTENT1', priority_orders),
    'CUODR': agg_wrapper('CUODR', priority_orders),
    'CENSEIGNE': agg_wrapper('CENSEIGNE', priority_orders),
    'MACMPROF': 'sum',
    'flux_annuel': 'sum',
    'pnb_annuel': 'sum',
    'pnb_an_rep': 'sum',
    'nb_op_annuel': 'sum',
    'compte epargne': 'sum',
    'compte epargne a echeance': 'sum',
    'compte à terme': 'sum',
    'prêt immo': 'sum',
    'compte titre': 'sum',
    'prêt perso': 'sum',
    'crédit fonctionnement': 'sum',
    'crédit équipement': 'sum',
    'engagement par signature': 'sum',
    'santé prévoyance': 'sum',
    'prevoyance_pro': 'sum',
    'dommage': 'sum',
    'assurance vie': 'sum',
    'Cartes de Paiement': 'sum',
    'carte_business': 'sum',
    'carte_affaire': 'sum',
    'Esprit Libre PRO': 'sum',
    'BNP NET Pro et Evolution': 'sum',
    'Contrat Commerçant': 'sum',
    'etebac': 'sum',
    'pee': 'sum',
    'perco': 'sum',
    'perm_mad': 'sum',
    'credit_inv': 'sum',
    'credit_bail': 'sum',
    'securite_pro': 'sum',
    'EL_pro': 'sum',
    'net_pro_evol': 'sum',
    'CA': 'sum',
    'nb_salaries': 'sum',
    'section': agg_wrapper('section', priority_orders),
    'code_sous_marche': agg_wrapper('code_sous_marche', priority_orders),
    'CCOTABNP': agg_wrapper('CCOTABNP', priority_orders),
    'anc_mois_crea': 'max',
    'MACMREP': 'sum',
    'MFTCREP': 'sum',
    'k_eq_restant_du': 'sum'
}

# Group and aggregate data for 'ER' segment
grouped_data_er = df_pro_unique_er.groupby('CCLIKPI').agg(agg_dict).reset_index()

# Group and aggregate data for the rest
grouped_data_rest = df_pro_unique_rest.groupby('CCLIKPI').agg(agg_dict).reset_index()

## Preparing the Target Variable

We read additional data files from 2018 to 2023 to prepare the target variable.

In [ ]:
# Read data files from 2018 to 2023
additional_files = [
    'data/STOCK_PROFILPRO_CLIENTS_122018.csv',
    '../domino/datasets/local/sp_2019_to_2023/STOCK_PROFILPRO_CLIENTS_122019.csv',
    '../domino/datasets/local/sp_2019_to_2023/STOCK_PROFILPRO_CLIENTS_122020.csv',
    '../domino/datasets/local/sp_2019_to_2023/STOCK_PROFILPRO_CLIENTS_122021.csv',
    '../domino/datasets/local/sp_2019_to_2023/STOCK_PROFILPRO_CLIENTS_122022.csv',
    '../domino/datasets/local/sp_2019_to_2023/STOCK_PROFILPRO_CLIENTS_062023.csv'
]

dataframes_additional = [pd.read_csv(file, sep=';', encoding='latin-1') for file in additional_files]

### Calculating Total PNB for Each ID

We calculate the total `pnb_annuel` (presumably a financial metric) for each `CCLIKPI` across the additional dataframes.

In [ ]:
# Function to calculate total pnb for each id
def calculate_pnb_for_each_id(og_df, dataframes, key='CCLIKPI', target_col='pnb_annuel'):
    # Get the unique ids from the original dataframe
    og_ids = set(og_df[key].unique())
    
    # Initialize a dictionary to store the total pnb for each id
    total_pnb_dict = {id: 0 for id in og_ids}
    
    # Iterate through each dataframe
    for df in dataframes:
        # Filter the dataframe to only include rows where the id is in the original dataframe
        filtered_df = df[df[key].isin(og_ids)]
        
        # Sum the pnb column for the filtered dataframe by id and add it to the total_pnb_dict
        for id, pnb_sum in filtered_df.groupby(key)[target_col].sum().items():
            total_pnb_dict[id] += pnb_sum
    
    return total_pnb_dict

### For 'ER' Segment

In [ ]:
# Calculate total pnb for each id in grouped_data_er
og = grouped_data_er
total_pnb_dict = calculate_pnb_for_each_id(og, dataframes_additional)

# Create a dataframe with the total_pnb
result_df = pd.DataFrame({
    'CCLIKPI': list(total_pnb_dict.keys()),
    'total_pnb': list(total_pnb_dict.values())
})

# Merge total_pnb with grouped_data_er
df_er = pd.merge(grouped_data_er, result_df, on='CCLIKPI', how='inner')

# Save the dataframe
df_er.to_csv('df_er.csv', index=False)

### For the Rest

In [ ]:
# Calculate total pnb for each id in grouped_data_rest
og = grouped_data_rest
total_pnb_dict = calculate_pnb_for_each_id(og, dataframes_additional)

# Create a dataframe with the total_pnb
result_df = pd.DataFrame({
    'CCLIKPI': list(total_pnb_dict.keys()),
    'total_pnb': list(total_pnb_dict.values())
})

# Merge total_pnb with grouped_data_rest
df_rest = pd.merge(grouped_data_rest, result_df, on='CCLIKPI', how='inner')

# Exclude rows where CENSEIGNE == 2
df_rest = df_rest[df_rest['CENSEIGNE'] != 2]

# Save the dataframe
df_rest.to_csv('df_rest.csv', index=False)

## Saving Final Dataframes

We save the final dataframes `df_er.csv` and `df_rest.csv` for further modeling.

---

**Note**: This notebook has been structured to provide a clear understanding of each step involved in the data preparation process. Each section includes explanations and code snippets to make the notebook easy to follow and aesthetically pleasing.